<div
  style="
    background-color: #f0f0f0;
    color:rgb(56, 56, 56);
    padding: 8px;
    display: flex;
    align-items: center;
    gap: 100px;
  "
>
  <img src="./images/brand.svg" style="max-height: 80px;">
  <strong>
    AI Saga: Data Science and Machine Learning</br>
    2.lab.2. Auto MPG Regression
  </strong>
</div>

### Background

You will continue working with the Auto MPG dataset, implementing regression models using scikit-learn.

The dataset includes:
- 398 instances
- 8 attributes including MPG, cylinders, displacement, horsepower, weight, acceleration, model year and origin

### Description

Your task is to implement regression models using scikit-learn to:

- Implement linear regression to predict MPG
- Implement polynomial regression with different degrees
- Compare the performance of linear vs polynomial models
- Visualize the fits for the most relevant feature

### Deliverables

- A *ipynb* (Jupyter Notebook) file called *auto-mpg-regression.ipynb*
- Visualizations comparing linear and polynomial fits
- Analysis of model performance for different polynomial degrees
- Use the provided template as a starting point


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = sns.load_dataset("mpg")

df = df.dropna()


df.head()

## Matriz de coorrelacioon 


Para saber qué variable predice mejor el MPG, calculamos correlaciones entre todas las variables numéricas. Esto pues  nos ayuda a encontrar automáticamente la característica qu es mas importante.



- elect_dtypes(include=[np.number]) filtra automáticamente solo las columnas numéricas
- corr() calcula la matriz de correlación de Pearson entre todas las variables
- sns.heatmap() visualiza la matriz con colores para identificar correlaciones fuertes
- abs().sort_values() ordena las correlaciones por valor absoluto para encontrar la más fuerte

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="RdBu_r", center=0)
plt.title("Matriz de Correlación")
plt.show()

 esta  Identifica automáticamente la variable más correlacionada

 La matriz nos revela que weight tiene la correlación más fuerte con MPG (-0.83), seguido por displacement (-0.81) y cylinders (-0.78). Esta correlación es negativa pero como se toma encunta el valor absoluto de puede indicar que los vehículos más pesados consumen más combustible, lo cual es lógico a mi parecer.

In [ ]:
mpg_correlations = correlation_matrix["mpg"].abs().sort_values(ascending=False)
most_relevant_feature = mpg_correlations.index[1]  # Excluir MPG mismo
print(
    f"Variable más correlacionada: {most_relevant_feature} ({mpg_correlations.iloc[1]:.3f})"
)


## Visualización de la relación principal

-  plt.scatter() crea un gráfico de dispersión para visualizar la relación entre dos variables
-  plt.hist() genera histogramas para ver la distribución de una variable individual
-  plt.subplot()  permite crear múltiples gráficos en una sola figura

Graficamos la relación entre weight y MPG para entender  mejor y visualmente

Se observa una clara relación que  no  es lineal entre weight y MPG, Los vehículos ligeros de menos de 2500 lb  muestran gran variabilidad en lo qeu respecta eficiencia 15-45 MPG segun la grafica  mientras que vehículos pesados sde mas de 4000 lbs están concentrados en bajos valores de MPG eentre 10 y 20 en base a eso se define que Existe un punto de inflexión que estaa redondeando on mejor dicho alrededor  de 3000 lbs donde la eficiencia empieza a caer.


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df[most_relevant_feature], df["mpg"], alpha=0.6)
plt.xlabel(most_relevant_feature.capitalize())
plt.ylabel("MPG")
plt.title(f"Relación entre {most_relevant_feature.capitalize()} y MPG")
plt.grid(True, alpha=0.3)
plt.show()



## Preprocesamiento con Scikit-Learn

La normalización aqui  es importante  porque las variables tienen escalas muy diferentes. Por ejemplo, weight puede estar en miles \ mientras que acceleration está en decenas por ende se hace el preprocesamitno:


- train_test_split() divide automáticamente los datos de forma aleatoria
- StandardScaler() calcula la media y desviación estándar del conjunto de entrenamiento con fit_transform() y aplica la misma transformación al conjunto de prueba con transform()


In [ ]:
X_multi = df[["cylinders", "displacement", "horsepower", "weight", "acceleration"]]
X_single = df[[most_relevant_feature]]
y = df["mpg"]

# División entrenamiento/prueba con scikit-learn
X_multi_train, X_multi_test, X_single_train, X_single_test, y_train, y_test = (
    train_test_split(X_multi, X_single, y, test_size=0.2, random_state=42)
)

# normalización con scikit-learn
scaler_multi = StandardScaler()
scaler_single = StandardScaler()

X_multi_train_scaled = scaler_multi.fit_transform(X_multi_train)
X_multi_test_scaled = scaler_multi.transform(X_multi_test)

X_single_train_scaled = scaler_single.fit_transform(X_single_train)
X_single_test_scaled = scaler_single.transform(X_single_test)

## Implementación de Regresión Lineal Multivariable

Utilizo scikit-learn para crear un modelo que usa múltiples características:


- LinearRegression() crea el objeto del modelo de regresión lineal
- fit() entrena el modelo calculando los coeficientes óptimos usando mínimos cuadrados
- predict() genera predicciones usando la ecuación lineal aprendida
- mean_squared_error() calcula el error promedio al cuadrado entre predicciones y valores reales
- r2_score() mide qué porcentaje de la varianza explica el modelo (0-1, donde 1 es perfecto)



In [ ]:
# Entrenar modelo de regresión lineal multivariable
linear_multi = LinearRegression()
linear_multi.fit(X_multi_train_scaled, y_train)

y_pred_linear_multi = linear_multi.predict(X_multi_test_scaled)
mse_linear_multi = mean_squared_error(y_test, y_pred_linear_multi)
r2_linear_multi = r2_score(y_test, y_pred_linear_multi)

print(
    f"Regresión lineal multivariable - MSE: {mse_linear_multi:.2f},  R^2: {r2_linear_multi:.3f}"
)

## Implementación de Regresión Lineal Univariable

Creo un modelo usando únicamente la característica más relevante (weight). Esto nos sirve de baseline para comparar con modelos más complejos.


In [ ]:
linear_single = LinearRegression()
linear_single.fit(X_single_train_scaled, y_train)

# Evaluar rendimiento
y_pred_linear_single = linear_single.predict(X_single_test_scaled)
mse_linear_single = mean_squared_error(y_test, y_pred_linear_single)
r2_linear_single = r2_score(y_test, y_pred_linear_single)

print(
    f"Regresión lineal univariable - MSE: {mse_linear_single:.2f}, R^2: {r2_linear_single:.3f}"
)

## Implementación de Regresión Polinomial con diferentes grados

>Me basé en el código propuesto en clase para implementar modelos polinomiales: 

- PolynomialFeatures(degree=degree) transforma automáticamente x en [x, x^2] según el grado
- fit_transform() aprende la transformación en entrenamiento y la aplica
- transform() aplica la misma transformación polinomial a los datos de prueba
- El loop for degree in degrees permite probar múltiples grados automáticamente
- Cada modelo se entrena con LinearRegression() pero sobre las características que ya estan  transformadas

básicamente, convertimos el problema no lineal en uno lineal al transformar las características esto por que un modelo de grado 2 puede aprender curvas, grado 3 curvas más complejas y asi.


In [ ]:
degrees = [1, 2, 3, 4, 5]
polynomial_results = {}

for degree in degrees:
    # Crear características polinomiales
    poly_features = PolynomialFeatures(degree=degree)
    X_poly_train = poly_features.fit_transform(X_single_train_scaled)
    X_poly_test = poly_features.transform(X_single_test_scaled)

    # Entrenar modelo
    poly_model = LinearRegression()
    poly_model.fit(X_poly_train, y_train)

    # Predicciones y métricas
    y_pred_poly = poly_model.predict(X_poly_test)
    mse_poly = mean_squared_error(y_test, y_pred_poly)
    r2_poly = r2_score(y_test, y_pred_poly)

    polynomial_results[degree] = {
        "model": poly_model,
        "poly_features": poly_features,
        "mse": mse_poly,
        "r2": r2_poly,
        "y_pred": y_pred_poly,
    }

    print(f"grado {degree} - MSE: {mse_poly:.2f}, R^2: {r2_poly:.3f}")

## Comparación del rendimiento de modelos

- pd.DataFrame() organiza todos los resultados en una tabla estructurada
- plt.subplots() crea múltiples (2) gráficos lado a lado para comparación
- bar() genera gráficos de barras para comparar métricas entre modelos
- tick_params(rotation=45) rota las etiquetas del eje x para mejor legibilidad


 Los resultados muestran que el modelo lineal multivariable supera un poco al univariable, pero ambos tienen limitaciones debido a lo no lineal de los datos. 
 
 Los modelos polinomiales de grado 2-3 presentan mejoras significativas en R^2 demostrando  que se  capturan mejor la curvatura natural de la relación weight-MPG.

In [ ]:
results_df = pd.DataFrame(
    {
        "Modelo": ["Lineal Multi", "Lineal Single"]
        + [f"Polinomial Grado {d}" for d in degrees],
        "MSE": [mse_linear_multi, mse_linear_single]
        + [polynomial_results[d]["mse"] for d in degrees],
        "R²": [r2_linear_multi, r2_linear_single]
        + [polynomial_results[d]["r2"] for d in degrees],
    }
)

print("=== Comparación de Modelos ===")
print(results_df.round(3))

# Visualiza la comparación
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.bar(results_df["Modelo"], results_df["MSE"], color="lightcoral")
ax1.set_title("Error Cuadrático Medio (MSE)")
ax1.set_ylabel("MSE")
ax1.tick_params(axis="x", rotation=45)

ax2.bar(results_df["Modelo"], results_df["R²"], color="lightblue")
ax2.set_title("Coeficiente de Determinación (R²)")
ax2.set_ylabel("R²")
ax2.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

## Visualización de ajustes para la característica más relevante

 múltiples subplots para comparar visualmente todos los modelos:


- np.linspace() genera puntos uniformemente que son espaciados para crear curvas de predicción
- plt.subplot(2, 3, i+1) crea una matriz  o cuadricula de 2x3 gráficos siguiendo el patrón del código propuesto en ,los ejemplos
- plt.scatter() muestra los datos reales de prueba
- plt.plot() dibuja la línea de predicción del modelo sobre los datos
- El loop permite automatizar la visualización de todos los grados polinomiales



- ### Grado 1 (Lineal): 
La línea recta no captura la curvatura natural de los datos, resultando en predicciones no muy conscias o por asi decrrlo nunpco pobres.

- ### Grado 2: 
Muestra una curva que sigue mejor el patrón de los datos, con mejora notable en R^2.

- ### Grado 3:

Ligera mejora adicional, capturando mejor los matices de la relación no lineal.

- ### Grados  4-5:
 Presenta oscilaciones en los extremos, indicando sobreajuste  en los datos de entrenamiento.


In [ ]:
plt.figure(figsize=(15, 10))

# Crear datos para curva suave de predicción
X_plot = np.linspace(X_single_train.min(), X_single_train.max(), 300).reshape(-1, 1)
X_plot_scaled = scaler_single.transform(X_plot)

for i, degree in enumerate([1, 2, 3, 4, 5]):
    plt.subplot(2, 3, i + 1)

    # datos de prueba
    plt.scatter(X_single_test, y_test, alpha=0.6, s=30)

    if degree == 1:
        # Predicción lineal
        y_plot = linear_single.predict(X_plot_scaled)
        mse_show = mse_linear_single
        r2_show = r2_linear_single
    else:
        # predicción polinomial
        poly_features = polynomial_results[degree]["poly_features"]
        X_plot_poly = poly_features.transform(X_plot_scaled)
        y_plot = polynomial_results[degree]["model"].predict(X_plot_poly)
        mse_show = polynomial_results[degree]["mse"]
        r2_show = polynomial_results[degree]["r2"]

    plt.plot(X_plot, y_plot, "r-", linewidth=2)
    plt.title(f"Grado {degree}\nMSE: {mse_show:.1f}, R²: {r2_show:.3f}")
    plt.xlabel(most_relevant_feature.capitalize())
    plt.ylabel("MPG")
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Identificación del mejor modelo

pandas para automatizar la selección del modelo óptimo:

- idxmax() encuentra automáticamente el índice del valor máximo en la columna R^2
- loc[] selecciona la fila completa correspondiente al mejor modelo
- Esto nos da una recomendación objetiva basada en métricas numéricas



In [ ]:
best_idx = results_df["R²"].idxmax()
best_model = results_df.loc[best_idx]

print(f"Modelo: {best_model['Modelo']}")
print(f"MSE: {best_model['MSE']:.2f}")
print(f"R^2: {best_model['R²']:.3f}")

## Conclusion final

El análisis revela que la regresión polinomial de grado 2-3 es mejor  para predecir MPG basado en el peso del vehículo

**Hallazgos**


1. Weight es la variable más predictiva con correlación con MPG
2. La relación es claramente no lineal, con un punto de inflexión alrededor de 3000 lbs
3. Modelos polinomiales grado 2-3 ofrecen el mejor balance entre precisión y simplicidad
4. Grados superiores (4-5) muestran sobreajuste sin beneficio realmente bueno o signbificativo
5. Variables múltiples mejoran  la predicción

